In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()


Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 https://developer.downlo

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-10-08 17:37:09--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.52MB/s    in 0.2s    

2022-10-08 17:37:10 (4.52 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   25933450| RJOVP071AVAJO|0439873800|      84656342|There Was an Old ...|           Books|          5|            0|          0|   N|                Y|          Five Stars|I love it and so ...|2015-08-31 00:00:00|
|         US|    1801372|R1ORGBETCDW3AI|1623953553|     729938122|      I Saw a Friend| 

### Create DataFrames to match tables

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   25933450| RJOVP071AVAJO|0439873800|      84656342|There Was an Old ...|           Books|          5|            0|          0|   N|                Y|          Five Stars|I love it and so ...|2015-08-31 00:00:00|
|         US|    1801372|R1ORGBETCDW3AI|1623953553|     729938122|      I Saw a Friend| 

In [6]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RJOVP071AVAJO|          5|            0|          0|   N|                Y|
|R1ORGBETCDW3AI|          5|            0|          0|   N|                Y|
| R7TNRFQAOUTX5|          5|            0|          0|   N|                Y|
|R2GANXKDIFZ6OI|          5|            0|          0|   N|                N|
|R2NYB6C3R8LVN6|          5|            2|          2|   N|                Y|
|R13U5PBJI1H94K|          2|            1|          1|   N|                N|
|R1H8UVH990F8VE|          5|            2|          2|   N|                N|
|R2MC0N30WZMRQ5|          5|            0|          0|   N|                Y|
| R2NO2HXK16Y4J|          5|            0|          0|   N|                Y|
|R245YIAVJK82ZL|          5|            0|          0|   N|     

In [8]:
 #Filter the vine_table to retrieve all votes where the total_votes >= to 20
 total_votes_df = vine_df.filter(vine_df.total_votes >= 20)
 total_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3T6LR4H11XRAN|          5|           26|         26|   N|                Y|
|R3QFNFMFWOQIBO|          3|           15|         25|   N|                Y|
|R24S2SV0ZIFIMV|          1|           41|         78|   N|                N|
|R2QAPWF1M5863L|          1|            5|         23|   N|                N|
|R3OCONKYE1A047|          5|           33|         38|   N|                N|
|R28WZIA84DUP6C|          2|            4|         43|   N|                N|
|R2U7YNJZ5JHWVD|          5|           23|         25|   N|                Y|
|R1XVYF74WCJKO3|          1|           47|         49|   N|                N|
|R3KKWNG5VABDYC|          5|           31|         32|   Y|                N|
|R3KK9MHFLY2BVW|          5|           21|         22|   N|     

In [31]:
from pyspark.sql import functions as F

#Create new dataframe to retrieve all helpful_votes/total_votes >= 50%
total_votes_df = total_votes_df.withColumn("Percentage", F.col("helpful_votes")/F.col("total_votes")*100)
total_votes_df = total_votes_df.withColumn ("Percentage", F.round(F.col("Percentage"),2))

#total_votes_df.show()

helpful_votes_df = total_votes_df.filter(total_votes_df.Percentage >= 50)
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+----------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|Percentage|
+--------------+-----------+-------------+-----------+----+-----------------+----------+
|R3T6LR4H11XRAN|          5|           26|         26|   N|                Y|     100.0|
|R3QFNFMFWOQIBO|          3|           15|         25|   N|                Y|      60.0|
|R24S2SV0ZIFIMV|          1|           41|         78|   N|                N|     52.56|
|R3OCONKYE1A047|          5|           33|         38|   N|                N|     86.84|
|R2U7YNJZ5JHWVD|          5|           23|         25|   N|                Y|      92.0|
|R1XVYF74WCJKO3|          1|           47|         49|   N|                N|     95.92|
|R3KKWNG5VABDYC|          5|           31|         32|   Y|                N|     96.88|
|R3KK9MHFLY2BVW|          5|           21|         22|   N|                N|     95.45|
|R2OEP8MGX5K370|     

In [32]:
paid_df = helpful_votes_df. filter(helpful_votes_df.vine == "Y")
paid_df.show() 

+--------------+-----------+-------------+-----------+----+-----------------+----------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|Percentage|
+--------------+-----------+-------------+-----------+----+-----------------+----------+
|R3KKWNG5VABDYC|          5|           31|         32|   Y|                N|     96.88|
|R397PIJYYVJ7PD|          4|           31|         32|   Y|                N|     96.88|
| RP6Y2CTIFUYD0|          2|           31|         32|   Y|                N|     96.88|
|R1FYRNM5U0S2VZ|          3|           20|         24|   Y|                N|     83.33|
| R1ME19UY4UNAO|          5|           53|         57|   Y|                N|     92.98|
|R1JW7ZHT2VLT1E|          3|           40|         45|   Y|                N|     88.89|
| R98UYXY43TURV|          4|           46|         50|   Y|                N|      92.0|
|R34VXM3EJDS78Z|          3|          374|        410|   Y|                N|     91.22|
|R1QX8UOW5JO6L0|     

In [33]:
unpaid_df = helpful_votes_df. filter(helpful_votes_df.vine == "N")
unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+----------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|Percentage|
+--------------+-----------+-------------+-----------+----+-----------------+----------+
|R3T6LR4H11XRAN|          5|           26|         26|   N|                Y|     100.0|
|R3QFNFMFWOQIBO|          3|           15|         25|   N|                Y|      60.0|
|R24S2SV0ZIFIMV|          1|           41|         78|   N|                N|     52.56|
|R3OCONKYE1A047|          5|           33|         38|   N|                N|     86.84|
|R2U7YNJZ5JHWVD|          5|           23|         25|   N|                Y|      92.0|
|R1XVYF74WCJKO3|          1|           47|         49|   N|                N|     95.92|
|R3KK9MHFLY2BVW|          5|           21|         22|   N|                N|     95.45|
|R2OEP8MGX5K370|          5|           30|         34|   N|                Y|     88.24|
| RTX63ENBKWCXF|     

In [34]:
#Determine total paid reviews
total_paid_reviews = paid_df.count()
print(f"Paid DataFrame review count: {total_paid_reviews}")

Paid DataFrame review count: 5012


In [37]:
#Determine number of 5-star reviews for paid
total_five_paid = paid_df.where(paid_df.star_rating == '5').count()
print(f"Paid 5-star review count: {total_five_paid}")

Paid 5-star review count: 2031


In [40]:
percentage_five_paid = round(total_five_paid/total_paid_reviews * 100, 2)
print(f"Paid 5-star review percentage: {percentage_five_paid}%")

Paid 5-star review percentage: 40.52%


In [41]:
#Determine total unpaid reviews
total_unpaid_reviews = unpaid_df.count()
print(f"Unpaid DataFrame review count: {total_unpaid_reviews}")

Unpaid DataFrame review count: 109297


In [43]:
#Determine number of 5-star reviews for unpaid
total_five_unpaid = unpaid_df.where(unpaid_df.star_rating == '5').count()
print(f"Unpaid 5-star review count: {total_five_unpaid}")

Unpaid 5-star review count: 49967


In [44]:
percentage_five_unpaid = round(total_five_unpaid/total_unpaid_reviews * 100, 2)
print(f"Unpaid 5-star review percentage: {percentage_five_unpaid}%")

Unpaid 5-star review percentage: 45.72%
